### 数据准备

In [118]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [119]:
user_features = pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/features/user_features.csv')
movie_features = pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/features/movie_features.csv')
ratings=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/ratings.csv')
movies=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/movies.csv')
users=pd.read_csv('/Users/bytedance/Desktop/MovieLens-Recommendation-System/data/ml-1m/users.csv')

### 查看数据情况

In [120]:
user_features.head()

,user_id,mean_rating,rating_std,rating_count,rating_min,rating_max,rating_strictness,rating_variability,Action_favorite_degree,Adventure_favorite_degree,Animation_favorite_degree,Children's_favorite_degree,Comedy_favorite_degree,Crime_favorite_degree,Documentary_favorite_degree,Drama_favorite_degree,Fantasy_favorite_degree,Film-Noir_favorite_degree,Horror_favorite_degree,Musical_favorite_degree,Mystery_favorite_degree,Romance_favorite_degree,Sci-Fi_favorite_degree,Thriller_favorite_degree,War_favorite_degree,Western_favorite_degree,Action_rating_cnt,Adventure_rating_cnt,Animation_rating_cnt,Children's_rating_cnt,Comedy_rating_cnt,Crime_rating_cnt,Documentary_rating_cnt,Drama_rating_cnt,Fantasy_rating_cnt,Film-Noir_rating_cnt,Horror_rating_cnt,Musical_rating_cnt,Mystery_rating_cnt,Romance_rating_cnt,Sci-Fi_rating_cnt,Thriller_rating_cnt,War_rating_cnt,Western_rating_cnt,favorite_genre,num_liked_genres
0,1,4.188679,0.680967,53,3,5,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.120690,0.017241,0.000000,0.181034,0.025862,0.000000,0.000000,0.120690,0.000000,0.051724,0.025862,0.025862,0.017241,0.000000,5,5,18,20,14,2,0,21,3,0,0,14,0,6,3,3,2,0,Drama,13
1,2,3.713178,1.001513,129,1,5,-0.131614,0.269719,0.194444,0.065972,0.000000,0.000000,0.086806,0.041667,0.000000,0.274306,0.003472,0.003472,0.006944,0.000000,0.010417,0.083333,0.059028,0.107639,0.052083,0.010417,56,19,0,0,25,12,0,79,1,1,2,0,3,24,17,31,15,3,Drama,14
2,3,3.901961,0.984985,51,1,5,-0.320396,0.252433,0.186992,0.203252,0.024390,0.024390,0.243902,0.000000,0.000000,0.065041,0.016260,0.000000,0.024390,0.008130,0.008130,0.040650,0.048780,0.040650,0.016260,0.048780,23,25,3,3,30,0,0,8,2,0,3,1,1,5,6,5,2,6,Comedy,15
3,4,4.190476,1.077917,21,1,5,-0.608912,0.257230,0.327586,0.103448,0.000000,0.017241,0.000000,0.017241,0.000000,0.103448,0.034483,0.000000,0.051724,0.000000,0.000000,0.034483,0.155172,0.068966,0.051724,0.034483,19,6,0,1,0,1,0,6,2,0,3,0,0,2,9,4,3,2,Action,12
4,5,3.146465,1.132699,198,1,5,0.435100,0.359991,0.088068,0.025568,0.011364,0.017045,0.159091,0.059659,0.017045,0.295455,0.000000,0.008523,0.028409,0.008523,0.022727,0.085227,0.042614,0.110795,0.017045,0.002841,31,9,4,6,56,21,6,104,0,3,10,3,8,30,15,39,6,1,Drama,17


In [121]:
movie_features.head()

,movie_id,title,genres,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_mean_rating,movie_rating_std,movie_rating_count,genre_purity
0,1,Toy Story (1995),Animation|Children's|Comedy,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.146846,0.852349,2077,0.333333
1,2,Jumanji (1995),Adventure|Children's|Fantasy,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.201141,0.983172,701,0.333333
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,3.016736,1.071712,478,0.500000
3,4,Waiting to Exhale (1995),Comedy|Drama,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2.729412,1.013381,170,0.500000
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.006757,1.025086,296,1.000000


In [122]:
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### 数据预处理

#### 1. 处理movie_features

In [123]:
print("用户评分数据情况:")
print(ratings['rating'].value_counts())

用户评分数据情况:
rating
4    348971
3    261197
5    226310
2    107557
1     56174
Name: count, dtype: int64


In [124]:
# 处理电影的标题
import re
pattern = re.compile(r'^(.*)\((\d+)\)$')

# 1. 去除掉特殊字符
title_map = {val:pattern.match(val).group(1) for ii,val in enumerate(set(movie_features['title']))}
movie_features['title'] = movie_features['title'].map(title_map)

# 2. 转换为数字字典
title_set=set()
for val in movie_features['title'].str.split():
    title_set.update(val)

title_set.add('<PAD>')
title2int={val: i for i, val in enumerate(title_set)}

# 3. 将电影title转成等长列表，长度为15
title_count=15
# 遍历movies_df['title']列中所有唯一的电影标题，对于每个电影标题，将其按照空格分割成单词，然后通过title2int字典将每个单词转换为对应的整数
title_map = {val: [title2int[row] for row in val.split()] for val in movie_features['title']}
for key in title_map:
    for cnt in range(title_count-len(title_map[key])):
        title_map[key].insert(len(title_map[key])+cnt, title2int['<PAD>'])
movie_features['title']=movie_features['title'].map(title_map)

In [155]:
pd.set_option('display.max_columns', None)
movie_features_numeric=movie_features.drop(['genres', 'title'], axis=1)
print("movie_features_numeric:", movie_features_numeric.shape)
movie_features_numeric.head()

movie_features_numeric: (3706, 23)


,movie_id,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,movie_mean_rating,movie_rating_std,movie_rating_count,genre_purity
0,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4.146846,0.852349,2077,0.333333
1,2,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3.201141,0.983172,701,0.333333
2,3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,3.016736,1.071712,478,0.500000
3,4,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2.729412,1.013381,170,0.500000
4,5,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3.006757,1.025086,296,1.000000


#### 2. 处理user_features

In [126]:
user_features.head()

,user_id,mean_rating,rating_std,rating_count,rating_min,rating_max,rating_strictness,rating_variability,Action_favorite_degree,Adventure_favorite_degree,Animation_favorite_degree,Children's_favorite_degree,Comedy_favorite_degree,Crime_favorite_degree,Documentary_favorite_degree,Drama_favorite_degree,Fantasy_favorite_degree,Film-Noir_favorite_degree,Horror_favorite_degree,Musical_favorite_degree,Mystery_favorite_degree,Romance_favorite_degree,Sci-Fi_favorite_degree,Thriller_favorite_degree,War_favorite_degree,Western_favorite_degree,Action_rating_cnt,Adventure_rating_cnt,Animation_rating_cnt,Children's_rating_cnt,Comedy_rating_cnt,Crime_rating_cnt,Documentary_rating_cnt,Drama_rating_cnt,Fantasy_rating_cnt,Film-Noir_rating_cnt,Horror_rating_cnt,Musical_rating_cnt,Mystery_rating_cnt,Romance_rating_cnt,Sci-Fi_rating_cnt,Thriller_rating_cnt,War_rating_cnt,Western_rating_cnt,favorite_genre,num_liked_genres
0,1,4.188679,0.680967,53,3,5,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.120690,0.017241,0.000000,0.181034,0.025862,0.000000,0.000000,0.120690,0.000000,0.051724,0.025862,0.025862,0.017241,0.000000,5,5,18,20,14,2,0,21,3,0,0,14,0,6,3,3,2,0,Drama,13
1,2,3.713178,1.001513,129,1,5,-0.131614,0.269719,0.194444,0.065972,0.000000,0.000000,0.086806,0.041667,0.000000,0.274306,0.003472,0.003472,0.006944,0.000000,0.010417,0.083333,0.059028,0.107639,0.052083,0.010417,56,19,0,0,25,12,0,79,1,1,2,0,3,24,17,31,15,3,Drama,14
2,3,3.901961,0.984985,51,1,5,-0.320396,0.252433,0.186992,0.203252,0.024390,0.024390,0.243902,0.000000,0.000000,0.065041,0.016260,0.000000,0.024390,0.008130,0.008130,0.040650,0.048780,0.040650,0.016260,0.048780,23,25,3,3,30,0,0,8,2,0,3,1,1,5,6,5,2,6,Comedy,15
3,4,4.190476,1.077917,21,1,5,-0.608912,0.257230,0.327586,0.103448,0.000000,0.017241,0.000000,0.017241,0.000000,0.103448,0.034483,0.000000,0.051724,0.000000,0.000000,0.034483,0.155172,0.068966,0.051724,0.034483,19,6,0,1,0,1,0,6,2,0,3,0,0,2,9,4,3,2,Action,12
4,5,3.146465,1.132699,198,1,5,0.435100,0.359991,0.088068,0.025568,0.011364,0.017045,0.159091,0.059659,0.017045,0.295455,0.000000,0.008523,0.028409,0.008523,0.022727,0.085227,0.042614,0.110795,0.017045,0.002841,31,9,4,6,56,21,6,104,0,3,10,3,8,30,15,39,6,1,Drama,17


In [127]:
# 处理用户最喜欢的电影类型
unique_genres = movies['genres'].str.get_dummies(sep='|').columns
genre_to_idx = {genre: idx for idx, genre in enumerate(unique_genres)}

user_features['favorite_genre_idx'] = user_features['favorite_genre'].map(
    lambda x: genre_to_idx.get(x, 0)  # 未知类型默认为0
)

In [128]:
print("user_features维度:", user_features.shape)
user_features_numeric=user_features.drop(['favorite_genre'], axis=1)
user_features_numeric.head()

user_features维度: (6040, 47)


,user_id,mean_rating,rating_std,rating_count,rating_min,rating_max,rating_strictness,rating_variability,Action_favorite_degree,Adventure_favorite_degree,Animation_favorite_degree,Children's_favorite_degree,Comedy_favorite_degree,Crime_favorite_degree,Documentary_favorite_degree,Drama_favorite_degree,Fantasy_favorite_degree,Film-Noir_favorite_degree,Horror_favorite_degree,Musical_favorite_degree,Mystery_favorite_degree,Romance_favorite_degree,Sci-Fi_favorite_degree,Thriller_favorite_degree,War_favorite_degree,Western_favorite_degree,Action_rating_cnt,Adventure_rating_cnt,Animation_rating_cnt,Children's_rating_cnt,Comedy_rating_cnt,Crime_rating_cnt,Documentary_rating_cnt,Drama_rating_cnt,Fantasy_rating_cnt,Film-Noir_rating_cnt,Horror_rating_cnt,Musical_rating_cnt,Mystery_rating_cnt,Romance_rating_cnt,Sci-Fi_rating_cnt,Thriller_rating_cnt,War_rating_cnt,Western_rating_cnt,num_liked_genres,favorite_genre_idx
0,1,4.188679,0.680967,53,3,5,-0.607115,0.162573,0.043103,0.043103,0.155172,0.172414,0.120690,0.017241,0.000000,0.181034,0.025862,0.000000,0.000000,0.120690,0.000000,0.051724,0.025862,0.025862,0.017241,0.000000,5,5,18,20,14,2,0,21,3,0,0,14,0,6,3,3,2,0,13,7
1,2,3.713178,1.001513,129,1,5,-0.131614,0.269719,0.194444,0.065972,0.000000,0.000000,0.086806,0.041667,0.000000,0.274306,0.003472,0.003472,0.006944,0.000000,0.010417,0.083333,0.059028,0.107639,0.052083,0.010417,56,19,0,0,25,12,0,79,1,1,2,0,3,24,17,31,15,3,14,7
2,3,3.901961,0.984985,51,1,5,-0.320396,0.252433,0.186992,0.203252,0.024390,0.024390,0.243902,0.000000,0.000000,0.065041,0.016260,0.000000,0.024390,0.008130,0.008130,0.040650,0.048780,0.040650,0.016260,0.048780,23,25,3,3,30,0,0,8,2,0,3,1,1,5,6,5,2,6,15,4
3,4,4.190476,1.077917,21,1,5,-0.608912,0.257230,0.327586,0.103448,0.000000,0.017241,0.000000,0.017241,0.000000,0.103448,0.034483,0.000000,0.051724,0.000000,0.000000,0.034483,0.155172,0.068966,0.051724,0.034483,19,6,0,1,0,1,0,6,2,0,3,0,0,2,9,4,3,2,12,0
4,5,3.146465,1.132699,198,1,5,0.435100,0.359991,0.088068,0.025568,0.011364,0.017045,0.159091,0.059659,0.017045,0.295455,0.000000,0.008523,0.028409,0.008523,0.022727,0.085227,0.042614,0.110795,0.017045,0.002841,31,9,4,6,56,21,6,104,0,3,10,3,8,30,15,39,6,1,17,7


#### 3. 查看数据情况

In [130]:
# 获取电影类型名
genres=movies['genres'].str.get_dummies(sep='|')
genre_columns=genres.columns
print(f"总共有{len(genre_columns)}个电影类型")

# 获取rating_cnt的列
genre_related_cols = [col for col in user_features.columns if any(genre in col for genre in genre_columns)]
print(len(genre_related_cols))

总共有18个电影类型
36


In [131]:
print("电影类型数为:", len(genre_columns))
print("电影数量有:", len(movie_features))
print("总的评价数有:", len(ratings))
print("总的用户数量为:", len(user_features))

# 不包含user_id, favorite_genre
user_numeric_cols = [
    'mean_rating', 'rating_std', 'rating_count', 'rating_min', 'rating_max',
    'rating_strictness', 'rating_variability', 'num_liked_genres', 'favorite_genre_idx'
] + [col for col in user_features.columns if col in genre_related_cols]
print("选定的用户数值型特征列数量为:", len(user_numeric_cols))

# 不包含movie_id, genres
movie_cols = [
    'movie_mean_rating', 'movie_rating_std', 'movie_rating_count', 'genre_purity'
] + [col for col in movie_features_numeric.columns if col in genre_columns]
print("选定的电影特征数量为:", len(movie_cols))

print("选定的用户数值型特征:", user_numeric_cols)
print("选定的电影特征:", movie_cols)
print("电影类型:", list(genre_columns))

电影类型数为: 18
电影数量有: 3706
总的评价数有: 1000209
用户数量为: 6040
选定的用户数值型特征列数量为: 45
选定的电影特征数量为: 22
选定的用户数值型特征: ['mean_rating', 'rating_std', 'rating_count', 'rating_min', 'rating_max', 'rating_strictness', 'rating_variability', 'num_liked_genres', 'favorite_genre_idx', 'Action_favorite_degree', 'Adventure_favorite_degree', 'Animation_favorite_degree', "Children's_favorite_degree", 'Comedy_favorite_degree', 'Crime_favorite_degree', 'Documentary_favorite_degree', 'Drama_favorite_degree', 'Fantasy_favorite_degree', 'Film-Noir_favorite_degree', 'Horror_favorite_degree', 'Musical_favorite_degree', 'Mystery_favorite_degree', 'Romance_favorite_degree', 'Sci-Fi_favorite_degree', 'Thriller_favorite_degree', 'War_favorite_degree', 'Western_favorite_degree', 'Action_rating_cnt', 'Adventure_rating_cnt', 'Animation_rating_cnt', "Children's_rating_cnt", 'Comedy_rating_cnt', 'Crime_rating_cnt', 'Documentary_rating_cnt', 'Drama_rating_cnt', 'Fantasy_rating_cnt', 'Film-Noir_rating_cnt', 'Horror_rating_cnt', 'Musical

In [132]:
print("movie_features特征列:", movie_features_numeric.columns)
print("user_features特征列:", user_features.columns)

movie_features特征列: Index(['movie_id', 'Action', 'Adventure', 'Animation', 'Children's', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror',
       'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western',
       'movie_mean_rating', 'movie_rating_std', 'movie_rating_count',
       'genre_purity'],
      dtype='object')
user_features特征列: Index(['user_id', 'mean_rating', 'rating_std', 'rating_count', 'rating_min',
       'rating_max', 'rating_strictness', 'rating_variability',
       'Action_favorite_degree', 'Adventure_favorite_degree',
       'Animation_favorite_degree', 'Children's_favorite_degree',
       'Comedy_favorite_degree', 'Crime_favorite_degree',
       'Documentary_favorite_degree', 'Drama_favorite_degree',
       'Fantasy_favorite_degree', 'Film-Noir_favorite_degree',
       'Horror_favorite_degree', 'Musical_favorite_degree',
       'Mystery_favorite_degree', 'Romance_favorite_degree',
       'Sci-Fi_favorite_degree', 'Thrill

### 创建映射字典

In [133]:
# 给定user_id的情况下，可以获取其特征值
user_id_to_feature = {} # key是user_id, value是用户特征 
for _, row in user_features.iterrows():
    try:
        user_id_to_feature[row['user_id']] = row[user_numeric_cols].values.astype(np.float32)
    except KeyError as e:
        print(f"Missing column in user_features: {e}")
        break
print("给定用户id的情况下，该用户的数值型特征数量为:",user_id_to_feature[1].shape) # 有45个特征（不包含user_id, favorite_genre）

给定用户id的情况下，该用户的数值型特征数量为: (45,)


In [134]:
# 给定movie_id的情况下，可以获取其特征值
movie_id_to_feature = {} # key是movie_id，value是电影特征
for _, row in movie_features_numeric.iterrows():
    try:
        movie_id_to_feature[row['movie_id']] = row[movie_cols].values
    except KeyError as e:
        print(f"Missing column in movie_features: {e}")
        break
print("给定电影id的情况下，该电影的特征数量为:", movie_id_to_feature[1].shape) # 有22个特征

给定电影id的情况下，该电影的特征数量为: (22,)


### 数据集类

In [135]:
class RatingDataset(Dataset):
    def __init__(self, data, user_id_to_feature, movie_id_to_feature):
        self.data = data
        self.user_id_to_feature = user_id_to_feature # user_id到feature的字典 
        self.movie_id_to_feature = movie_id_to_feature # movie_id到feature的字典
        
        # 获取favorite_genre的映射
        self.genre_to_idx = {genre: idx for idx, genre in enumerate(genre_columns)}
        self.num_genres = len(genre_columns)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        """
        给定idx的情况下，可以得到某一条评分数据，包含user_id, movie_id, rating
        """
        user_id = self.data['user_id'][idx]
        movie_id=self.data['movie_id'][idx]
        rating=self.data['rating'][idx]
        
        # 获取数值特征
        user_feature = self.user_id_to_feature[user_id] 
        movie_feature = self.movie_id_to_feature[movie_id] 
        
        # 处理favorite_genre（转换为one-hot）
        favorite_genre = user_features[user_features['user_id'] == user_id]['favorite_genre'].values[0]
        genre_idx = self.genre_to_idx.get(favorite_genre, 0)
        genre_onehot = torch.zeros(self.num_genres)
        genre_onehot[genre_idx] = 1
        
        # 合并所有用户特征
        user_feature = np.concatenate([
            user_feature,  
            genre_onehot.numpy() 
        ])
        
        normalized_rating = (rating - 1) / 5.0 # 最大值为5， 最小值为1
        return (
            torch.FloatTensor(user_feature),
            torch.FloatTensor(movie_feature),
            torch.FloatTensor([normalized_rating])
        )

### 划分训练集和验证集&创建数据加载器

In [137]:
merged_df_1 = pd.merge(ratings, movie_features_numeric, left_on='movie_id', right_on='movie_id', how='inner')
# 再根据user_id将上一步结果和user_features_numeric合并
final_merged_df = pd.merge(merged_df_1, user_features_numeric, left_on='user_id', right_on='user_id', how='inner')

In [139]:
train_data, test_data=train_test_split(final_merged_df, test_size=0.2, random_state=42)

train_dataset = RatingDataset(train_data, user_id_to_feature, movie_id_to_feature)
test_dataset = RatingDataset(test_data, user_id_to_feature, movie_id_to_feature)

In [140]:
print("训练集样本数量:", len(train_dataset))
print("验证集样本数量:", len(test_dataset))

训练集样本数量: 800167
验证集样本数量: 200042


In [141]:
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# print("数据索引范围:", train_dataset.data.min(), train_dataset.data.max())

### 模型架构

In [115]:
class DualTowerModel(nn.Module):
    def __init__(self, user_feature_dim, movie_feature_dim, embedding_dim=128):
        super(DualTowerModel, self).__init__()
        
        # 用户塔
        self.user_tower = nn.Sequential(
            nn.Linear(user_feature_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(128, embedding_dim//2),
            nn.ReLU()
        )
        
        # 电影塔
        self.movie_tower = nn.Sequential(
            nn.Linear(movie_feature_dim, embedding_dim),
            nn.ReLU(),
            nn.Linear(128, embedding_dim//2),
            nn.ReLU()
        )
        
        # 评分预测头
        self.rating_head = nn.Sequential(
            nn.Linear(embedding_dim, embedding_dim//2),
            nn.ReLU(),
            nn.Linear(embedding_dim//2, 1),
            nn.Sigmoid()  # 输出在0-1之间
        )
    
    def forward(self, user_features, movie_features):
        user_embedding = self.user_tower(user_features)
        movie_embedding = self.movie_tower(movie_features)
        combined = torch.cat([user_embedding, movie_embedding], dim=1)
        return self.rating_head(combined).squeeze()

### 初始化模型

In [143]:
sample_user, sample_movie, _ = train_dataset[1] # 用户特征，电影特征，评分
user_feature_dim = sample_user.shape[0] # 用户特征情况
movie_feature_dim = sample_movie.shape[0] # 电影特征情况
print("user_feature_dim:", user_feature_dim)
print("movie_feature_dim:", movie_feature_dim)
model = DualTowerModel(user_feature_dim, movie_feature_dim)

user_feature_dim: 63
movie_feature_dim: 22


In [173]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import time
import os

# 1. 训练配置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 模型参数
user_feature_dim = sample_user.shape[0]
movie_feature_dim = sample_movie.shape[0]
model = DualTowerModel(user_feature_dim, movie_feature_dim)
print(model)


Using device: cpu
DualTowerModel(
  (user_tower): Sequential(
    (0): Linear(in_features=63, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (movie_tower): Sequential(
    (0): Linear(in_features=22, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
  )
  (rating_head): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=1, bias=True)
    (3): Sigmoid()
  )
)


In [183]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
import os

def train_dual_tower_model(model, train_loader, test_loader, num_epochs=10, learning_rate=0.001, weight_decay=1e-5, device='cpu'):
    # 移动模型到指定设备
    model = model.to(device)
    
    # 定义损失函数和优化器
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # 学习率调度器
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)
    
    # 用于跟踪最佳模型
    best_val_loss = float('inf')
    best_model_path = '/Users/bytedance/Desktop/MovieLens-Recommendation-System/models/best_dual_tower_model.pth'
    
    # 训练循环
    for epoch in range(num_epochs):
        # 训练模式
        model.train()
        train_loss = 0.0
        for user_features, movie_features, ratings in train_loader:
            # 移动数据到设备
            user_features = user_features.to(device)
            movie_features = movie_features.to(device)
            ratings = ratings.to(device)
            
            # 前向传播
            outputs = model(user_features, movie_features)
            loss = criterion(outputs, ratings)
            
            # 反向传播和优化
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item() * user_features.size(0)
        
        train_loss /= len(train_loader.dataset)
        
        # 验证模式
        model.eval()
        val_loss = 0.0
        val_predictions = []
        val_targets = []
        
        with torch.no_grad():
            for user_features, movie_features, ratings in test_loader:
                user_features = user_features.to(device)
                movie_features = movie_features.to(device)
                ratings = ratings.to(device)
                
                outputs = model(user_features, movie_features)
                loss = criterion(outputs, ratings)
                
                val_loss += loss.item() * user_features.size(0)
                val_predictions.extend(outputs.cpu().numpy())
                val_targets.extend(ratings.cpu().numpy())
        
        val_loss /= len(test_loader.dataset)
        val_rmse = np.sqrt(mean_squared_error(val_targets, val_predictions))
        
        # 更新学习率
        scheduler.step(val_loss)
        
        # 打印日志
        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val RMSE: {val_rmse:.4f}')
        
        # 保存最佳模型
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            os.makedirs(os.path.dirname(best_model_path), exist_ok=True)
            torch.save(model.state_dict(), best_model_path)
            print(f'Saved best model with Val Loss: {best_val_loss:.4f}')
    
    # 加载最佳模型
    model.load_state_dict(torch.load(best_model_path))
    return model, best_val_loss

def main():
    # 设置设备
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # 假设已定义 train_loader, test_loader, 和 model
    # 这些已在您的数据处理代码中定义
    batch_size = 256
    num_epochs = 10
    learning_rate = 0.001
    weight_decay = 1e-5
    
    # 实例化模型（使用您提供的 user_feature_dim 和 movie_feature_dim）
    user_feature_dim = 63  # 从您的输出：45（数值特征）+ 18（genre one-hot）
    movie_feature_dim = 22  # 从您的输出
    model = DualTowerModel(user_feature_dim, movie_feature_dim)
    
    # 训练模型
    trained_model, best_val_loss = train_dual_tower_model(
        model,
        train_loader,
        test_loader,
        num_epochs=num_epochs,
        learning_rate=learning_rate,
        weight_decay=weight_decay,
        device=device
    )
    
    print(f'Training completed. Best validation loss: {best_val_loss:.4f}')

In [184]:
main()

Using device: cpu


AttributeError: module 'torch._functorch.eager_transforms' has no attribute 'grad_and_value'